In [343]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# predictors
from sklearn import svm
from sklearn import linear_model

import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Step 1: Preprocessing

In [344]:
combine = [train, test]

In [345]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [346]:
# train_desc = train.describe()
# train.dtypes.to_dict()
len(test.columns)

80

Now looking at all features to find the best ones:

## Find the relevant features
Here we find the standard deviation of all features concerning the SalePrice.

Assumption: If a feature has a low standard deviation here, it correleates strongly with SalePrice and is thus important.

But we can only do this for features where the datatype is something else than double. Features with the datatype double naturally have a low standard deviation because most values are different for them.

We will not combine features for now.
So features with the following can be excluded:
- low count (where not many data rows have values for)
- many distinct values (but will be used later)
- high standard deviation mean

In [350]:
# inserting new Areas for testing purposes
for dataset in combine:
    print(dataset[["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea"]])
    dataset["TotalPorch"] = dataset["WoodDeckSF"]+dataset["OpenPorchSF"]+dataset["EnclosedPorch"]+dataset["3SsnPorch"]+dataset["ScreenPorch"]].sum()
    dataset["ExpensiveArea"] = dataset["3SsnPorch"]+dataset["PoolArea"]
print(dataset["ExpensiveArea"])

      WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea
0              0           61              0          0            0         0
1            298            0              0          0            0         0
2              0           42              0          0            0         0
3              0           35            272          0            0         0
4            192           84              0          0            0         0
5             40           30              0        320            0         0
6            255           57              0          0            0         0
7            235          204            228          0            0         0
8             90            0            205          0            0         0
9              0            4              0          0            0         0
10             0            0              0          0            0         0
11           147           21              0        

In [325]:
good_features = []

In [326]:
feature_inspection_table = pd.DataFrame(data={'feature':[], 'weighted_std_mean':[], 'count':[], 'distinct_values':[], 'datatype':[]})
feature_inspection_table.set_index("feature")
for feature, datatype in train.dtypes.iteritems(): #select_dtypes(exclude=['float64','int64']):
    if feature == "Id":
        continue
    ov = train.groupby(feature)['SalePrice'].agg(['count','std'])
    # TODO: weighted mean
    mean = ov['std'].mean()
    ov["std_sum"] = ov["count"] * ov["std"]
    count = ov["count"].sum()
    distinct_values = ov.shape[0]
    weighted_mean =  ov["std_sum"].sum() / count
    feature_inspection_table = feature_inspection_table.append({"feature":feature,"weighted_std_mean":weighted_mean,"count":count,"distinct_values":distinct_values,'datatype':datatype},ignore_index=True)

c:\python37\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [327]:
excluded_features = {"count":[],"distinct_values":[]}
min_count = 750
max_distinct_values = 50
for feature, values in feature_inspection_table.iterrows():
    if values["count"] <= min_count:
        excluded_features["count"] += [values["feature"]]
    if values["distinct_values"] >= max_distinct_values:
        excluded_features["distinct_values"].append(values["feature"])
# remove rows where count is too low
reduced_table = feature_inspection_table[feature_inspection_table["count"] > min_count]
# remove rows with too few distinct_values
reduced_table = reduced_table[reduced_table["distinct_values"] < max_distinct_values]

print("Excluded features: {}".format(excluded_features))
reduced_table.sort_values(by='weighted_std_mean').head(25)

Excluded features: {'count': ['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'TotalPorch', 'ExpensiveArea'], 'distinct_values': ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'SalePrice']}


,feature,weighted_std_mean,count,distinct_values,datatype
16,OverallQual,40742.442685,1460.0,10.0,int64
11,Neighborhood,49039.324204,1460.0,25.0,object
60,GarageCars,52063.413232,1460.0,5.0,int64
52,KitchenQual,54517.157369,1460.0,4.0,object
26,ExterQual,54678.135679,1460.0,4.0,object
29,BsmtQual,55565.221640,1423.0,4.0,object
48,FullBath,61001.593173,1460.0,4.0,int64
53,TotRmsAbvGrd,62242.638262,1460.0,12.0,int64
59,GarageFinish,64593.201079,1379.0,3.0,object
0,MSSubClass,65776.128304,1460.0,15.0,int64


In [328]:
# Looking at the correlations:
c = pd.DataFrame(train.corr()["SalePrice"])
c = c.iloc[c["SalePrice"].abs().argsort()]
c = c.iloc[::-1]
c

,SalePrice
ExpensiveArea,NaN
ExpensiveArea,NaN
SalePrice,1.000000
OverallQual,0.790982
GrLivArea,0.708624
GarageCars,0.640409
GarageArea,0.623431
TotalBsmtSF,0.613581
1stFlrSF,0.605852
FullBath,0.560664


**What is done here:**
- for every feature:
    - the values of the feature get grouped together and a new column for the standard deviation is created
    - the average of the standard deviation is listed as "std_mean" in the table above
    - the weighted mean of the standard deviation (taking the count of each value into account) is listed under "weighted_mean" above

Lower values in the "weighted_mean" column indicate more important features as the correlation between these features and the price is higher.

Features with too few entries or too many distinct values have been filtered from the above list but might still be useful.

In the following we will look at the most important features and take them into account in the following.

## GarageCars, FullBath
The table above shows that all of these features have a relatively low standard deviation and will thus be used without further explaination.

They are integers, so can simply be added to the features that are taken into account without further preprocessing.

In [145]:
# See how many missing values we have:
print("Train: --- \n{}".format(train.isnull().sum()[["GarageCars","FullBath"]]))
print("Test: --- \n{}".format(test.isnull().sum()[["GarageCars","FullBath"]]))

Train: --- 
GarageCars    0
FullBath      0
dtype: int64
Test: --- 
GarageCars    1
FullBath      0
dtype: int64


Thus the test set has a row where GarageCars is NaN. Lets just insert 0 there:

In [146]:
for dataset in combine:
    dataset["FullBath"] = dataset["FullBath"].fillna(value=0)
    dataset["GarageCars"] = dataset["GarageCars"].fillna(value=0)

In [147]:
good_features += ["GarageCars","FullBath"]

## Neighborhood
These features also show a good correlation, but they are strings and must be converted to numbers.

In [148]:
# train[['Neighborhood', 'SalePrice']].groupby(['Neighborhood'], as_index=False).mean().sort_values(by='SalePrice', ascending=False)
neighborhood = train.groupby('Neighborhood')['SalePrice'].agg(['count','mean','std']).sort_values(by='mean', ascending=False).reset_index()
neighborhood.head()

,Neighborhood,count,mean,std
0,NoRidge,41,335295.317073,121412.658640
1,NridgHt,77,316270.623377,96392.544954
2,StoneBr,25,310499.000000,112969.676640
3,Timber,38,242247.447368,64845.651549
4,Veenker,11,238772.727273,72369.317959


We will just replace the neighborhood's name with the mean house price in that neighborhood. The standard deviation is quite high but we will ignore this for now.

In [149]:
# series to dict:
neighborhood_dict = neighborhood[["Neighborhood","mean"]].set_index('Neighborhood').to_dict()['mean']

In [150]:
for dataset in combine:
    dataset['Neighborhood_mean'] = dataset['Neighborhood'].map(neighborhood_dict)
    
good_features.append('Neighborhood_mean')

## KitchenQual, ExterQual, BsmtQual
Like Neighborhood, these features have a good correlation. But here we have only 4 possible values. BsmtQual is special because some houses have no basement.

The following values are possible:
- Ex: Excellent
- Gd: Good
- TA: Typical / Average
- Fa: Fair
- Po: Poor

For the basement, there is also NoValue

In [165]:
qual_features = ["KitchenQual", "ExterQual", "BsmtQual"]
# See how many missing values we have:
print(train.isnull().sum()[qual_features])
print(test.isnull().sum()[qual_features])

KitchenQual    0
ExterQual      0
BsmtQual       0
dtype: int64
KitchenQual    0
ExterQual      0
BsmtQual       0
dtype: int64


In [152]:
for feature in qual_features:
    ov = train.groupby(feature)['SalePrice'].agg(['count','mean','std']).sort_values(by='mean', ascending=False).reset_index()
    print(ov.head())

  KitchenQual  count           mean            std
0          Ex    100  328554.670000  120862.942573
1          Gd    586  212116.023891   64020.176702
2          TA    735  139962.511565   38896.280336
3          Fa     39  105565.205128   36004.254037
  ExterQual  count           mean            std
0        Ex     52  367360.961538  116401.264200
1        Gd    488  231633.510246   71188.873899
2        TA    906  144341.313466   42471.815703
3        Fa     14   87985.214286   39826.918794
  BsmtQual  count           mean            std
0       Ex    121  327041.041322  113563.237392
1       Gd    618  202688.478964   58092.364284
2       TA    649  140759.818182   43483.265311
3       Fa     35  115692.028571   34469.667278


The simplest thing to do is to simply map the quality strings to values:
- Ex: 5
- Gd: 4
- TA: 3
- Fa: 2
- Po: 1

The question is what to do with the missing values for the basement.
For Now I will fill in just a 3 because that seems to be the most common value.

In [169]:
qual_dict = {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}
for dataset in combine:
    for feature in qual_features:
        dataset[feature] = dataset[feature].map(qual_dict)
        dataset[feature] = dataset[feature].fillna(value=3)
good_features += qual_features

## Features that were removed because of count
Now we can look at the features that were removed because too many rows used them:

In [206]:
for feature in excluded_features["count"]:
    l = train.groupby(feature)['SalePrice'].agg(['count','mean','std']).sort_values(by='mean', ascending=False).reset_index()
    print(l)

  Alley  count           mean           std
0  Pave     41  168000.585366  38370.375243
1  Grvl     50  122219.080000  34780.780734
  PoolQC  count    mean            std
0     Ex      2  490000  360624.458405
1     Fa      2  215500   48790.367902
2     Gd      3  201990   63441.392639
   Fence  count           mean           std
0  GdPrv     59  178927.457627  56757.237563
1  MnPrv    157  148751.089172  66096.069463
2   GdWo     54  140379.314815  53592.585059
3   MnWw     11  134286.363636  21781.805376
  MiscFeature  count           mean           std
0        TenC      1  250000.000000           NaN
1        Gar2      2  170750.000000  27223.611076
2        Shed     49  151187.612245  51642.728056
3        Othr      2   94000.000000  55154.328933


From this, the only feature that seems usable is Alley. The other features have either a very low count or a very high variance.

## Features removed because of the distinct values

In [208]:
excluded_features["distinct_values"]

['LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'SalePrice']

**There are the following different datatypes in this group:**
- Area Sizes:
    - LotArea
    - MasVnrArea: Masonry veneer area in square feet
    - WoodDeckSF: Wood deck area in square feet
    - OpenPorchSF: Open porch area in square feet
    - EnclosedPorch: Enclosed porch area in square feet
    - 3SsnPorch: Three season porch area in square feet
    - ScreenPorch: Screen porch area in square feet
    - PoolArea: Pool area in square feet
    - GrLivArea: Above grade (ground) living area square feet
    - 1stFlrSF: First Floor square feet
    - 2ndFlrSF: Second floor square feet
    - BsmtFinSF1: Type 1 finished square feet
    - BsmtFinSF2: Type 2 finished square feet
    - BsmtUnfSF: Unfinished square feet of basement area
    - TotalBsmtSF: Total square feet of basement area
    - LotFrontage: Linear feet of street connected to property
- Age:
    - YearBuilt
    - YearRemodAdd
    - GarageYrBlt

Now to find out which of these Correlate with the Price.

### Age

In [217]:
train.corr()["SalePrice"]

Id              -0.021917
MSSubClass      -0.084284
LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
OverallCond     -0.077856
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtFinSF2      -0.011378
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
LowQualFinSF    -0.025606
GrLivArea        0.708624
BsmtFullBath     0.227122
BsmtHalfBath    -0.016844
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr    -0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch   -0.128578
3SsnPorch        0.044584
ScreenPorch      0.111447
PoolArea         0.092404
MiscVal         -0.021190
MoSold           0.046432
YrSold          -0.028923
SalePrice        1.000000
Name: SalePr

## Create the sets and normalize

In [170]:
# for now (simplicity):
Kaggle_test = test[good_features]
X = train[good_features]
y = train["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(good_features)

['GarageCars', 'FullBath', 'Neighborhood_mean', 'KitchenQual', 'ExterQual', 'BsmtQual', 'KitchenQual', 'ExterQual', 'BsmtQual', 'KitchenQual', 'ExterQual', 'BsmtQual', 'KitchenQual', 'ExterQual', 'BsmtQual']


# Step 2: test different models and find the best one

## Linear SVR

clf = svm.SVR(kernel = 'linear')
clf.fit(X_train,y_train)
clf_predictions = clf.predict(X_test)
absolute_error = mean_absolute_error(y_test,clf_predictions)
squared_error = mean_squared_error(y_test,clf_predictions)
print(absolute_error)
print(squared_error)

## Ridge Regression

In [171]:
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train,y_train)
reg_predictions = reg.predict(X_test)
absolute_error = mean_absolute_error(y_test,reg_predictions)
squared_error = mean_squared_error(y_test,reg_predictions)
print(absolute_error)
print(squared_error)

# predict
print(Kaggle_test.head())
print(X_test.head())
final_predictions = reg.predict(Kaggle_test)
# z = pd.DataFrame(data=final_predictions,columns=["SalePrice"])
z = pd.DataFrame(final_predictions, index=test["Id"], columns=['SalePrice'])
# z["Id"] = test["Id"]
# z.set_index("Id")
print(z.head())
z.to_csv("my_predictions.csv")

33131.99017774048
2591095580.524899
   GarageCars  FullBath  Neighborhood_mean  KitchenQual  ExterQual  BsmtQual  \
0         1.0         1      145847.080000          3.0        3.0       3.0   
1         1.0         1      145847.080000          3.0        3.0       3.0   
2         2.0         2      192854.506329          3.0        3.0       3.0   
3         2.0         2      192854.506329          3.0        3.0       3.0   
4         2.0         2      310499.000000          3.0        3.0       3.0   

   KitchenQual  ExterQual  BsmtQual  KitchenQual  ExterQual  BsmtQual  \
0          3.0        3.0       3.0          3.0        3.0       3.0   
1          3.0        3.0       3.0          3.0        3.0       3.0   
2          3.0        3.0       3.0          3.0        3.0       3.0   
3          3.0        3.0       3.0          3.0        3.0       3.0   
4          3.0        3.0       3.0          3.0        3.0       3.0   

   KitchenQual  ExterQual  BsmtQual  
0     